In [1]:
import torch
from torch import nn 
from torch.utils.data import DataLoader, TensorDataset
from torchvision import io
import os
import matplotlib.pyplot as plt 
import numpy as np 

/home/yuning/anaconda3/envs/torch/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = ("cuda" if torch.cuda.is_available else "cpu")
print(device)

cuda


In [3]:
file_path= "/home/yuning/DL/Monument/Train"
file_type = [ os.path.join(file_path,image) for image in os.listdir(file_path)  ]
print(f"There are {len(file_type)} types of argument in the path")
print(f"A sample of file is {file_type[0]}")

There are 3 types of argument in the path
A sample of file is /home/yuning/DL/Monument/Train/basic


In [4]:
sample_path = [os.path.join(file_type[0],i) for i in os.listdir(file_type[0])][0]
print(sample_path)
channels,width,height = io.read_image(sample_path).size()
knsize = [5,3]
feature_dims = [32,64,128]
zdim = 32
print(channels,height,width)

/home/yuning/DL/Monument/Train/basic/Aug-134-0.jpg
3 640 360


In [5]:
from src import nn
g_e = nn.g_e(height,width,channels,device,knsize,zdim,feature_dims)
print(g_e.convnet.eval)
print(g_e.mlp.eval)

<bound method Module.eval of Sequential(
  (conv1): Conv2d(3, 32, kernel_size=(5, 5), stride=(2, 2), padding=(3, 3))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(2, 2))
)>
<bound method Module.eval of Sequential(
  (flatten): Flatten()
  (dense1): Linear(in_features=7372800, out_features=128, bias=True)
  (elu1): ELU(alpha=1.0)
  (dense2): Linear(in_features=128, out_features=32, bias=True)
  (elu2): ELU(alpha=1.0)
)>


In [6]:
x_sample = io.read_image(sample_path).float().to(device)
pred = g_e.convnet(x_sample)
# pred_out = g_e(x_sample)
print(pred.detach().cpu().size())
# print(pred.detach().cpu().size())

torch.Size([128, 47, 82])


In [ ]:
Tensors = []
for imag_type in file_type:
    image_files = [os.path.join(imag_type,image_name) for image_name in os.listdir(imag_type)]
    sub_tensors = [io.read_image(file_imag)/255 for file_imag in image_files]
    Tensors.append(sub_tensors)

In [ ]:
AllTensor = []
for tensors in Tensors:
    AllTensor.append( torch.stack([  i for i in tensors]))

All_Tensor = torch.concat(AllTensor)
print(All_Tensor.shape)

In [ ]:
Tensordataset = TensorDataset(All_Tensor)

In [ ]:
torch.manual_seed(1024)
train_dl = DataLoader(Tensordataset,batch_size=4,shuffle=True)